In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/Machine Learning/"
#file_name = "cleanedvehicles.csv"
file_name1 = "cleanedvehicles_rev1.csv"

In [ ]:
df_pip=pd.read_csv(file_path+'cleanedvehicles_rev2.csv')

In [ ]:
# Now I am going to do pipeline
import scipy
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
#Because region has 403 variables, and it is overlaped with latitude and longitude, We can try one test without it.
df_pip_noregion=df_pip.copy()
df_pip_noregion.drop(['region'], axis=1, inplace=True)

In [ ]:
df_pip_noregion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254324 entries, 0 to 254323
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         254324 non-null  int64  
 1   year          254324 non-null  int64  
 2   manufacturer  254324 non-null  object 
 3   condition     254324 non-null  object 
 4   cylinders     254324 non-null  object 
 5   fuel          254324 non-null  object 
 6   odometer      254324 non-null  int64  
 7   title_status  254324 non-null  object 
 8   transmission  254324 non-null  object 
 9   drive         254324 non-null  object 
 10  type          254324 non-null  object 
 11  state         254324 non-null  object 
 12  lat           254324 non-null  float64
 13  long          254324 non-null  float64
dtypes: float64(2), int64(3), object(9)
memory usage: 27.2+ MB


In [ ]:
df_pip = df_pip[df_pip['price'] > 10000]
df_pip = df_pip[df_pip['price'] < 20000]
# Rounded ['odometer'] to 5000
df_pip['odometer'] = df_pip['odometer'] // 5000
df_pip = df_pip[df_pip['year'] > 2000]

In [ ]:
df_pip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83808 entries, 1 to 254322
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        83808 non-null  object 
 1   price         83808 non-null  int64  
 2   year          83808 non-null  int64  
 3   manufacturer  83808 non-null  object 
 4   condition     83808 non-null  object 
 5   cylinders     83808 non-null  object 
 6   fuel          83808 non-null  object 
 7   odometer      83808 non-null  int64  
 8   title_status  83808 non-null  object 
 9   transmission  83808 non-null  object 
 10  drive         83808 non-null  object 
 11  type          83808 non-null  object 
 12  state         83808 non-null  object 
 13  lat           83808 non-null  float64
 14  long          83808 non-null  float64
dtypes: float64(2), int64(3), object(10)
memory usage: 10.2+ MB


In [ ]:
categorical_vars = ['region','manufacturer', 'condition', 'cylinders',
            'fuel', 'title_status', 'transmission', 'drive', 
            'type', 'state']

categorical_vars_noregion = ['manufacturer', 'condition', 'cylinders',
            'fuel', 'title_status', 'transmission', 'drive', 
            'type', 'state']

discrete_vars = ['year']

continuous_vars = ['odometer', 'lat','long']

dep_var = ['price']

In [ ]:
# we first try all the attributes including region
cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))

cat_pipe = Pipeline([cat_ohe_step])
cat_transformers = [('cat', cat_pipe, categorical_vars)]



num_scl_step = ('scl', MinMaxScaler())

num_pipe = Pipeline([num_scl_step])
num_transformers = [('num', num_pipe, discrete_vars + continuous_vars)]


ct = ColumnTransformer(transformers=cat_transformers + num_transformers)
ct.fit(df_pip[categorical_vars + discrete_vars + continuous_vars])
X = ct.transform(df_pip[categorical_vars + discrete_vars + continuous_vars])
y = df_pip[['price']].values

In [ ]:
X.shape
# y.shape

(83808, 541)

In [ ]:
# Then we can exclude region
cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))

cat_pipe = Pipeline([cat_ohe_step])
cat_transformers = [('cat', cat_pipe, categorical_vars_noregion)]



num_scl_step = ('scl', MinMaxScaler())

num_pipe = Pipeline([num_scl_step])
num_transformers = [('num', num_pipe, discrete_vars + continuous_vars)]


ct = ColumnTransformer(transformers=cat_transformers + num_transformers)
ct.fit(df_pip_noregion[categorical_vars_noregion + discrete_vars + continuous_vars])
X_noregion = ct.transform(df_pip_noregion[categorical_vars_noregion + discrete_vars + continuous_vars])
y_noregion = df_pip_noregion[['price']].values

Basedline: Linear Regression

In [ ]:
import numpy as np 
import pandas as pd 
from scipy.stats import randint
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from pandas import set_option
plt.style.use('ggplot') # nice plots

from sklearn import model_selection
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.linear_model import LinearRegression # to apply the Linear regression
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold # for cross validation
from sklearn.model_selection import GridSearchCV # for tuning parameter
from sklearn.model_selection import RandomizedSearchCV  # Randomized search on hyper parameters.
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_noregion,y_noregion,test_size = 0.3, random_state=0) #split train and test

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
LR = LinearRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_train)

In [ ]:
LR.score(X_train, y_train)

0.0007012648297526347

In [ ]:
r2_score(y_pred, y_train)

-1408.6230731160306

r2_score is still extremely low and linear regression model doesn't fit this dataset.

Linear SVR Model

Because linear regression model doesn't score well, I'm going to use Linear SVR Model to see if the modelling will improve the result. Linear SVR is a similar to SVM method. Its also builds on kernel functions but is appropriate for unsupervised learning.

In [ ]:
from sklearn.svm import SVR, LinearSVR

In [ ]:
linear_svr = LinearSVR()
linear_svr.fit(X_train, y_train)
y_pred = linear_svr.predict(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
linear_svr.score(X_train, y_train)

-5.988746784302279e-05

In [ ]:
r2_score(y_pred, y_train)

-38495566.22345348

It indicates that it's worse than linear regression model.

ExtraTree Regressor

ExtraTreesRegressor implements a meta estimator that fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The default values for the parameters controlling the size of the trees (e.g. max_depth, min_samples_leaf, etc.) lead to fully grown and unpruned trees which can potentially be very large on some data sets. To reduce memory consumption, the complexity and size of the trees should be controlled by setting those parameter values. 

In extremely randomized trees, randomness goes one step further in the way splits are computed. As in random forests, a random subset of candidate features is used, but instead of looking for the most discriminative thresholds, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This usually allows to reduce the variance of the model a bit more, at the expense of a slightly greater increase in bias.

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor 

In [ ]:
etr = ExtraTreesRegressor()
etr.fit(X_train, y_train.ravel())
y_pred = etr.predict(X_train)

In [ ]:
etr.score(X_train, y_train)

0.7607682360803311

In [ ]:
r2_score(y_pred, y_train)

0.6855392318255917